In [1]:
import os
import sys
import numpy as np
from orquestra.qml.models.tn.mps import (
    MPS,
    AMPS,
    EqualityConstrainedMps,
    OneSiteDMRG,
    RandCircuitInitializer,
    TwoSiteDMRG,
)



from orquestra.qml.constraints import AxbEqualityConstraint
from orquestra.qml.data_loaders import BarsAndStripesDataLoader, new_data_loader
from orquestra.qml.trainers import SimpleTrainer
from orquestra.qml.data_loaders import new_data_loader
import torch
from orquestra.qml.models.samplers.jx import RandomCardinalitySampler 
from orquestra.qml.data_loaders import new_data_loader


  Activating project at `~/workspace/repos/orquestra-qml-core/src/python/orquestra/qml/jl/QML.jl`


# Simple Example 



In [25]:
np.hstack((np.ones(8), np.zeros(8)))

array([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.])

In [2]:
n_x=8
n_y=8
# constraint = AxbEqualityConstraint(np.matrix(np.hstack((np.ones(8), np.zeros(8))), np.hstack((np.zeros(8), np.ones(8)))),np.ones(1, 1))
constraint1 = AxbEqualityConstraint(np.hstack((np.ones(8), np.zeros(8))),np.ones(1))
constraint2 = AxbEqualityConstraint(np.hstack((np.zeros(8), np.ones(8))),np.ones(1))
# filter out those that don't meet equality constraint

init_sample_first= RandomCardinalitySampler(16//2,1).generate(4)
init_sample_second= RandomCardinalitySampler(16//2,1).generate(4)


initial_samples_list = []




for first, second in zip(init_sample_first,init_sample_second):
    initial_samples_list.append(list(first)+list(second))

init_sample_array = np.asarray(initial_samples_list)



In [3]:
init_sample_array.shape[0]

4

In [3]:
dl=new_data_loader(torch.tensor(init_sample_array),batch_size=-1).shuffle(123456)

In [4]:
n_sites = 16
mps_gen = MPS(
    [2 for _ in range(n_sites)],
    initializer=RandCircuitInitializer(
        bond_dim=32,
        element_type="Float32",
    ),
    scheme=OneSiteDMRG(),
    output_type="Int32",
)

# assert len(mps_gen.tensors) == n_sites, "MPS must have n_sites tensors"
# assert (
#     len(mps_gen.bond_dimensions) == n_sites - 1
# ), "MPS must have n_sites-1 bond dimensions"

SimpleTrainer().train(
    mps_gen,
    dl,
    2,
)

samples = mps_gen.generate(10, 10)

Epoch 1 / 2:   0%|          | 0/1 [00:00<?, ?steps/s]

TypeError: Julia: MethodError: no method matching train_on_batch(::ITensors.MPS, ::PythonCall.PyArray{Int64, 2, true, false, Int64}, ::PythonCall.PyArray{PythonCall.Py, 0, true, true, PythonCall.UnsafePyObject}, ::Main.QML.TN.OneSiteDMRG)
Closest candidates are:
  train_on_batch(::ITensors.MPS, ::PythonCall.PyArray{<:Integer}, !Matched::PythonCall.PyArray{<:AbstractFloat}, ::T) where T<:Main.QML.TN.Scheme at ~/workspace/repos/orquestra-qml-core/src/python/orquestra/qml/jl/QML.jl/src/TN/mps.jl:101
  train_on_batch(!Matched::Main.QML.TN.AMPS, ::PythonCall.PyArray{<:Integer}, !Matched::PythonCall.PyArray{<:AbstractFloat}) at ~/workspace/repos/orquestra-qml-core/src/python/orquestra/qml/jl/QML.jl/src/TN/amps.jl:86

In [7]:
np.sum(samples,axis=1)

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4])